In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math
import re
from collections import Counter

In [ ]:
# Load sorted_sentences from file
with open('/content/curriculum_sorted.txt', 'r', encoding='utf-8') as f:
    sorted_sentences = [line.strip() for line in f.readlines()]
print(f"Loaded {len(sorted_sentences)} sorted sentences.")

curriculum_sentences = sorted_sentences

Loaded 447213 sorted sentences.


In [ ]:
curriculum_sentences = sorted_sentences  # from previous notebook
random_sentences = pd.read_csv('/content/telugu_books.csv')['text'].tolist()
child_sentences = pd.read_csv('/content/child_directed_telugu_1Lakh.csv')['sentence'].tolist()


In [ ]:
print(f"Curriculum: {len(curriculum_sentences)} sentences")
print(f"Random: {len(random_sentences)} sentences")
print(f"Child-directed: {len(child_sentences)} sentences")

Curriculum: 447213 sentences
Random: 25793 sentences
Child-directed: 100000 sentences


In [ ]:
def tokenize(sentence):
    if not isinstance(sentence, str):
        sentence = str(sentence)
    return sentence.split()

# Clean sentences properly
def clean_sentences(sent_list):
    clean = []
    for s in sent_list:
        if isinstance(s, str) and s.strip() != '':
            clean.append(s)
    return clean

In [ ]:
curriculum_sentences = clean_sentences(curriculum_sentences)
random_sentences = clean_sentences(random_sentences)
child_sentences = clean_sentences(child_sentences)

In [ ]:
def tokenize(sentence):
    if not isinstance(sentence, str):
        sentence = str(sentence)
    return re.findall(r'\w+', sentence)

In [ ]:
def clean_sentences(sent_list):
    clean = []
    for s in sent_list:
        if isinstance(s, str) and s.strip() != '':
            clean.append(s)
    return clean

In [ ]:
curriculum_sentences = curriculum_sentences[:500]
random_sentences = random_sentences[:100]
child_sentences = child_sentences[:500]

In [ ]:
print(f"New dataset sizes: Curriculum {len(curriculum_sentences)}, Random {len(random_sentences)}, Child-directed {len(child_sentences)}")

New dataset sizes: Curriculum 500, Random 100, Child-directed 500


In [ ]:
word_counter = Counter()
for s in (curriculum_sentences + random_sentences + child_sentences):
    word_counter.update(tokenize(s))

In [ ]:
MIN_FREQ = 5
vocab = [word for word, freq in word_counter.items() if freq >= MIN_FREQ]
word2idx = {w: i+1 for i, w in enumerate(vocab)}  # 0 = padding
idx2word = {i: w for w, i in word2idx.items()}
vocab_size = len(word2idx) + 1
print(f"Final Vocab size after filtering: {vocab_size}")

Final Vocab size after filtering: 790


In [ ]:
def encode(sent):
    return [word2idx[w] for w in tokenize(sent) if w in word2idx]

In [ ]:
class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, heads=4, layers=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=layers)
        self.fc = nn.Linear(embed_dim, vocab_size)

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [ ]:
def train_model(train_sentences, epochs=2):
    model = SimpleTransformer(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    model.train()

    for epoch in range(epochs):
        total_loss = 0
        random.shuffle(train_sentences)
        for sent in train_sentences:
            ids = encode(sent)
            if len(ids) < 2:
                continue
            input_ids = torch.tensor([ids[:-1]], device=device)
            target_ids = torch.tensor([ids[1:]], device=device)
            optimizer.zero_grad()
            output = model(input_ids)
            loss = criterion(output.view(-1, vocab_size), target_ids.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_sentences):.4f}")
    return model

In [ ]:
def calculate_perplexity(model, sentences):
    model.eval()
    total_loss = 0
    total_count = 0
    criterion = nn.CrossEntropyLoss(ignore_index=0, reduction='sum')
    with torch.no_grad():
        for sent in sentences:
            ids = encode(sent)
            if len(ids) < 2:
                continue
            input_ids = torch.tensor([ids[:-1]], device=device)
            target_ids = torch.tensor([ids[1:]], device=device)
            output = model(input_ids)
            loss = criterion(output.view(-1, vocab_size), target_ids.view(-1))
            total_loss += loss.item()
            total_count += len(ids) - 1
    ppl = math.exp(total_loss / total_count)
    return ppl

In [ ]:
print("Training on Curriculum")
curr_model = train_model(curriculum_sentences)
print("Training on Random")
rand_model = train_model(random_sentences)
print("Training on Child-directed")
child_model = train_model(child_sentences)

Training on Curriculum


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 2.3911
Epoch 2, Loss: 2.5406
Training on Random


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1, Loss: 4.2377
Epoch 2, Loss: 4.1454
Training on Child-directed
Epoch 1, Loss: 3.0184
Epoch 2, Loss: 2.9218


In [ ]:
curr_ppl = calculate_perplexity(curr_model, curriculum_sentences[:1000])
rand_ppl = calculate_perplexity(rand_model, random_sentences[:1000])
child_ppl = calculate_perplexity(child_model, child_sentences[:1000])

In [ ]:
print(f"Curriculum Perplexity: {curr_ppl:.2f}")
print(f"Random Perplexity: {rand_ppl:.2f}")
print(f"Child-directed Perplexity: {child_ppl:.2f}")

Curriculum Perplexity: 13.47
Random Perplexity: 62.64
Child-directed Perplexity: 18.07


In [ ]:
minimal_pairs = [
    ("అమ్మాయిలు బడికి వెళ్తున్నారు.", "అమ్మాయిలు బడికి వెళ్తుంది."),
    ("అతను పాలు తాగాడు.", "అతను పాలు తాగింది."),
    ("పిల్లలు ఆడుతున్నారు.", "పిల్లలు ఆడుతోంది."),
    ("అతడు స్కూల్‌కి వెళ్లాడు.", "అతడు స్కూల్‌కి వెళ్తాడు."),
    ("అమ్మ బడికి వెళ్లింది.", "అమ్మ బడి వెళ్లింది."),
    ("ఆమె పుస్తకం చదువుతుంది.", "ఆమె పుస్తకం చదువుతాడు."),
    ("నేను పాలు తాగాను.", "పాలు నేను తాగాను."),
]

In [ ]:
for correct, wrong in minimal_pairs:
    c_ids = encode(correct)
    w_ids = encode(wrong)
    if len(c_ids) < 2 or len(w_ids) < 2:
        continue
    input_c = torch.tensor([c_ids[:-1]], device=device)
    target_c = torch.tensor([c_ids[1:]], device=device)
    input_w = torch.tensor([w_ids[:-1]], device=device)
    target_w = torch.tensor([w_ids[1:]], device=device)
    c_loss = nn.CrossEntropyLoss(ignore_index=0)(curr_model(input_c).view(-1, vocab_size), target_c.view(-1))
    w_loss = nn.CrossEntropyLoss(ignore_index=0)(curr_model(input_w).view(-1, vocab_size), target_w.view(-1))
    print(f"Minimal pair: Correct loss={c_loss.item():.4f}, Wrong loss={w_loss.item():.4f}")

Minimal pair: Correct loss=7.2329, Wrong loss=7.9594
Minimal pair: Correct loss=4.9048, Wrong loss=4.7248
Minimal pair: Correct loss=6.4096, Wrong loss=7.3881
Minimal pair: Correct loss=6.8856, Wrong loss=6.9573
Minimal pair: Correct loss=7.7698, Wrong loss=8.6271
Minimal pair: Correct loss=9.0869, Wrong loss=9.2155
Minimal pair: Correct loss=4.7681, Wrong loss=4.9624
